In [2]:
import operator
import pandas as pd
import matplotlib.pylab as plt
import itertools

_ = np.seterr(invalid='ignore') # Pandas PITA Nan printing

import bounos.ChartBuilders as cb
import bounos.Analyses.Trust as Trust

import os

def map_level(df, dct, level=0):
    index = df.index
    index.set_levels([[dct.get(item, item) for item in names] if i==level else names
                      for i, names in enumerate(index.levels)], inplace=True)


cb.latexify(columns=0.5, factor=1)

phys_keys = ['INDD','INHD','Speed']
comm_keys = ['ADelay','ARXP','ATXP','RXThroughput','TXThroughput','PLR']
observer = 'Bravo'
target = 'Alfa'
n_nodes = 6
n_metrics = 9
par=False

results_path = "/home/bolster/src/aietes/results/Malicious Behaviour Trust Comparison-2015-07-20-17-47-53"

fig_basedir = "/home/bolster/src/thesis/papers/active/16_AAMAS/img"


var_rename_dict = {'CombinedBadMouthingPowerControl':'MPC',
 'CombinedSelfishTargetSelection':'STS',
 'CombinedTrust':'Fair',
 'Shadow':'Shadow',
 'SlowCoach':'SlowCoach'}

metric_rename_dict = {
    'ADelay': "$Delay$",
    'ARXP': "$P_{RX}$",
    'ATXP': "$P_{TX}$",
    'RXThroughput': "$T^P_{RX}$",
    'TXThroughput': "$T^P_{TX}$",
    'PLR': '$PLR$',
    'INDD': '$INDD$',
    'INHD': '$INHD$',
    'Speed': '$Speed$'
}
inv_map = lambda d: {v: k for k, v in d.items()}


assert os.path.isdir(fig_basedir)
def categorise_dataframe(df):
    # Categories work better as indexes
    for obj_key in df.keys()[df.dtypes == object]:
        try:
            df[obj_key] = df[obj_key].astype('category')
        except TypeError:
            print("Couldn't categorise {}".format(obj_key))
            pass
    return df

def non_zero_rows(df):
    return df[~(df==0).all(axis=1)]

from IPython.display import display
from collections import defaultdict
tree = lambda: defaultdict(tree)


                        
def calc_correlations_from_weights(weights):
    def calc_correlations(base, comp, index=0):
        dp_r = (comp / base).reset_index()
        return dp_r.corr()[index][:-1]
    _corrs = {}
    for base, comp in itertools.permutations(weights.keys(), 2):
        _corrs[(base, comp)] = \
            calc_correlations(weights[base],
                              weights[comp])

    corrs = pd.DataFrame.from_dict(_corrs).T.rename(columns=metric_rename_dict)
    map_level(corrs, var_rename_dict, 0)
    map_level(corrs, var_rename_dict, 1)
    corrs.index.set_names(['Control', 'Misbehaviour'], inplace=True)
    return corrs

%matplotlib inline 


/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
with pd.get_store(results_path + '.h5') as store:
    trust_observations = store.trust.xs('Bravo',level='observer', drop_level=False).dropna()

In [ ]:
unweighted_trust_perspectives = Trust.generate_node_trust_perspective(trust_observations,
                                                                    metric_weights=None,
                                                                    par=par)
map_level(unweighted_trust_perspectives, var_rename_dict)

In [4]:
with pd.HDFStore('/dev/shm/shared.h5',  mode='r') as store:
    feat_weights = categorise_dataframe(non_zero_rows(store.joined_feats).T)
    corrs = calc_correlations_from_weights(store.joined_target_weights)
    signed_feat_weights = feat_weights.rename(index=metric_rename_dict)*corrs.apply(np.sign).T 

In [5]:
signed_feat_weights

base             MPC                                     STS            \
comp             STS      Fair    Shadow SlowCoach       MPC      Fair   
$Delay$    -0.031665  0.038869 -0.026059  0.015367 -0.031517 -0.148742   
$INDD$     -0.019956 -0.013972 -0.123166 -0.212052  0.019981  0.160675   
$INHD$     -0.032802  0.018010 -0.022223 -0.090656 -0.032909  0.201226   
$PLR$       0.010224  0.010274  0.010910 -0.010690  0.010121  0.004023   
$P_{RX}$   -0.375582 -0.403282 -0.209257  0.044620  0.375899  0.114900   
$P_{TX}$   -0.462091 -0.469263 -0.528959 -0.345701 -0.461830 -0.003945   
$Speed$     0.026955  0.018448  0.025188  0.252233 -0.026945  0.118238   
$T^P_{RX}$ -0.025762  0.013789 -0.030117  0.017043  0.025834  0.137678   
$T^P_{TX}$ -0.014965 -0.014094 -0.024120  0.011638  0.014963  0.110572   

base                                Fair                                \
comp          Shadow SlowCoach       MPC       STS    Shadow SlowCoach   
$Delay$    -0.036319 -0.024497 -0.038773  0.148663  0.032953  0.021658   
$INDD$      0.293382  0.233893  0.013979 -0.159791 -0.265500 -0.209930   
$INHD$      0.073167  0.192667 -0.018023  0.202239  0.077043  0.146141   
$PLR$      -0.002413  0.001810  0.010030  0.003961  0.002170  0.001961   
$P_{RX}$   -0.412340 -0.197243 -0.403428 -0.114824 -0.426081 -0.225865   
$P_{TX}$    0.002387  0.001829 -0.469355 -0.003983  0.002171 -0.001956   
$Speed$     0.112927  0.328538  0.018508 -0.118680 -0.112843 -0.373331   
$T^P_{RX}$ -0.040210 -0.010451 -0.013769  0.137686 -0.043293  0.010271   
$T^P_{TX}$  0.026855 -0.009073 -0.014136  0.110173  0.037945  0.008886   

base          Shadow                               SlowCoach            \
comp             MPC       STS      Fair SlowCoach       MPC       STS   
$Delay$    -0.026094  0.036444 -0.033100 -0.024302 -0.015376 -0.024489   
$INDD$     -0.123049 -0.293687 -0.265048  0.146166  0.212022  0.234076   
$INHD$      0.022169 -0.073164 -0.077391 -0.240267 -0.090423  0.192654   
$PLR$      -0.010933 -0.002381 -0.002181 -0.002151  0.010932  0.001795   
$P_{RX}$    0.209260 -0.411918  0.426152  0.141112  0.044447  0.197366   
$P_{TX}$    0.529027 -0.002358  0.002178  0.002176  0.345662  0.001791   
$Speed$    -0.025187  0.113024  0.112725  0.391193 -0.252572  0.328301   
$T^P_{RX}$ -0.030102 -0.040159  0.043308  0.028008  0.016943 -0.010478   
$T^P_{TX}$  0.024177  0.026863 -0.037917  0.024625  0.011624  0.009048   

base                            
comp            Fair    Shadow  
$Delay$     0.021727 -0.024306  
$INDD$     -0.210230  0.145875  
$INHD$      0.145978  0.240342  
$PLR$      -0.001914 -0.002164  
$P_{RX}$    0.225977 -0.141574  
$P_{TX}$    0.001945  0.002180  
$Speed$     0.373015  0.391048  
$T^P_{RX}$  0.010305 -0.028018  
$T^P_{TX}$  0.008908  0.024491

In [ ]:
weighted_trust_perspectives = {}
for k,w in feat_weights.to_dict().items():
    if key[0]!='Fair':
        continue
    weighted_trust_perspectives[k] = Trust.generate_node_trust_perspective(trust_observations,
                                                                    metric_weights=pd.Series(w),
                                                                    par=par)
for key,trust in weighted_trust_perspectives.items():
    map_level(weighted_trust_perspectives[key],var_rename_dict)

In [ ]:
ewma = lambda f: pd.stats.moments.ewma(f,span=4)

ewma(unweighted_trust_perspectives.unstack('var')['Alfa'].xs('Bravo', level='observer')).boxplot()
plt.title("Unweighted")
plt.show()
for key,trust in weighted_trust_perspectives.items():
    if key[0]!='Fair':
        continue
    ax = ewma(trust.unstack('var')['Alfa'].xs('Bravo', level='observer')).boxplot(return_type='axes')
    title="Weighted {}".format('-'.join(key))
    ax.set_title(title)
    plt.show()


In [ ]:
with pd.get_store(results_path + "/outliers.bkup.h5") as store:       
    mdts = store.get('meandeltaCombinedTrust_2')
metrics = np.split(mdts.keys(),[4])[1]

from IPython.display import display
tops = {}
for k,v in mdts.groupby(['var']):
    tops[k]=v.sort(0, ascending=False)[metrics].head(1).values[0].astype(np.float64)

opti = pd.DataFrame.from_dict(tops, orient='index')
opti.columns = metrics
opti

reload(Trust)
signed_weighted_trust_perspectives = {}
for k,w in tops.items():
    print k
    signed_weighted_trust_perspectives[var_rename_dict.get(k)] = Trust.generate_node_trust_perspective(trust_observations,
                                                                    metric_weights=pd.Series(w),
                                                                    par=par)
for key,trust in signed_weighted_trust_perspectives.items():
    map_level(signed_weighted_trust_perspectives[key],var_rename_dict)

In [ ]:
reload(Trust)
signed_weighted_trust_perspectives = {}
for k,w in signed_feat_weights.rename(inv_map(metric_rename_dict)).to_dict().items():
    if k[0]!='Fair':
        continue
    signed_weighted_trust_perspectives[k] = Trust.generate_node_trust_perspective(trust_observations,
                                                                    metric_weights=pd.Series(w),
                                                                    par=par)
for key,trust in signed_weighted_trust_perspectives.items():
    map_level(signed_weighted_trust_perspectives[key],var_rename_dict)
    
bad_result = signed_weighted_trust_perspectives[('Fair','SlowCoach')].xs(0, level='run').xs('SlowCoach', level='var')

good_result = pd.stats.moments.ewma(signed_weighted_trust_perspectives[('Fair','MPC')].xs(0, level='run').xs('MPC', level='var'),
                      span=4)

In [ ]:
%timeit assess_result(signed_weighted_trust_perspectives[('Fair','MPC')])[0]

In [ ]:
%timeit np.subtract(*map(np.nanmean,np.split(signed_weighted_trust_perspectives[('Fair','MPC')].values, [1], axis=1)))

In [ ]:
feat_weights

In [ ]:
                  
#results = assess_dt_results(signed_weighted_trust_perspectives)
results = assess_results(signed_weighted_trust_perspectives)

results

In [ ]:
pd.DataFrame(results, index=['mean','std'])

In [ ]:
inv_signed_weighted_trust_perspectives = {}
for k,w in signed_feat_weights.items():
    inv_signed_weighted_trust_perspectives[k] = Trust.generate_node_trust_perspective(trust_observations,
                                                                    metric_weights=-pd.Series(w),
                                                                    par=par)
for key,trust in inv_signed_weighted_trust_perspectives.items():
    map_level(inv_signed_weighted_trust_perspectives[key],var_rename_dict)

In [ ]:
#signed_weighted_trust_perspectives[(u'Fair', u'MPC')]

In [ ]:
signed_feat_weights.rename(inv_map(metric_rename_dict)).T.describe()

In [ ]:
ewma(unweighted_trust_perspectives.unstack('var')['Alfa'].xs('Bravo', level='observer')).boxplot()
plt.title("Unweighted")
plt.show()
for key,trust in signed_weighted_trust_perspectives.items():
    if key[1]!='Fair':
        continue
    ax = ewma(trust.unstack('var')['Alfa'].xs('Bravo', level='observer')).boxplot(return_type='axes')
    title="Weighted {}".format('-'.join(key))
    ax.set_title(title)
    plt.show()

In [ ]:
#pd.DataFrame.from_records((signed_feat_weights[(u'Fair', u'MPC')],signed_feat_weights[(u'Fair', u'Shadow')])).plot(kind='bar')

In [ ]:
tf = trust_observations.xs('Alfa',level='target')
tf.columns.name='metrics'
tf.stack().swaplevel('var','metrics').unstack('var').groupby(level='metrics').boxplot()

In [ ]:
from itertools import islice

def split_every(n, iterable):
    i = iter(iterable)
    piece = list(islice(i, n))
    while piece:
        yield piece
        piece = list(islice(i, n))

fig, axes = plt.subplots(3, 3, sharey=False, sharex=True)

for i,row in enumerate(split_every(3, tf.stack().swaplevel('var','metrics').unstack('var').groupby(level='metrics'))):
    for j,(k,grp) in enumerate(row):
        grp.boxplot(ax=axes[i][j])
        axes[i][j].set_title(k)

In [ ]:
_=unweighted_trust_perspectives.unstack('var')['Alfa'].xs('Bravo', level='observer').boxplot()
plt.title("Unweighted")
plt.show()
for key,trust in inv_signed_weighted_trust_perspectives.items():
    if key[0]!='Fair':
        continue
    ax = trust.unstack('var')['Alfa'].xs('Bravo', level='observer').boxplot(return_type='axes')
    title="Inverted Weighted {}".format('-'.join(key))
    ax.set_title(title)
    plt.show()
    ax.get_figure().savefig(os.path.join(fig_basedir,"{}.png".format(title.replace(' ',''))))

In [ ]:
pd.DataFrame.from_dict(feat_weights).T.xs("Fair",0).plot(kind='bar')

In [ ]:
pd.DataFrame.from_dict(signed_feat_weights).T.xs("Fair",0).plot(kind='bar')

In [ ]:
(-pd.DataFrame.from_dict(signed_feat_weights)).T.xs("Fair",0).plot(kind='bar')